In [357]:
# import libraries
import numpy as np 
import pandas as pd
import string
from matplotlib import pyplot as plt

In [358]:
#Import data
df = pd.read_csv("https://raw.githubusercontent.com/monicafar147/classification-predict-streamlit-template/master/climate-change-belief-analysis/train.csv")
df1 = pd.read_csv("https://raw.githubusercontent.com/monicafar147/classification-predict-streamlit-template/master/climate-change-belief-analysis/test.csv")

In [359]:
# set tweetid as index
df_new = df1.set_index('tweetid')

In [360]:
# clean the train csv
df_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df['message'] = df['message'].replace(to_replace = df_url, value = subs_url, regex = True)
df['message'] = df['message'].str.lower()

In [361]:
# function to remove punctuations
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [362]:
df['message'] = df['message'].apply(remove_punctuation)

In [363]:
# clean the test csv
df_test = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_urltest = r'url-web'
df_new['message'] = df_new['message'].replace(to_replace = df_test, value = subs_urltest, regex = True)
df_new['message'] = df_new['message'].str.lower()

In [364]:
# function to remove punctuations
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [365]:
df_new['message'] = df_new['message'].apply(remove_punctuation)

In [366]:
df

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,its not like we lack evidence of anthropogenic...,126103
2,2,rt rawstory researchers say we have three year...,698562
3,1,todayinmaker wired 2016 was a pivotal year in...,573736
4,1,rt soynoviodetodas its 2016 and a racist sexis...,466954
...,...,...,...
15814,1,rt ezlusztig they took down the material on gl...,22001
15815,2,rt washingtonpost how climate change could be ...,17856
15816,0,notiven rt nytimesworld what does trump actual...,384248
15817,-1,rt sara8smiles hey liberals the climate change...,819732


In [367]:
# split into train and test
from sklearn.model_selection import train_test_split

X = df['message']  
y = df['sentiment']
X_unseen = df_new['message'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [368]:
 con = pd.concat([X_train, y_train], axis=1)
 con

,message,sentiment
14201,why do some people still doubt global warming ...,1
14693,rt trialwatcher1 drmartyfox sheriffclarke loud...,-1
3745,rt natgeochannel watch beforetheflood right he...,1
15273,rt greenpeace what if climate change is just a...,1
10324,rt emorwee in which i casually speculate that ...,1
...,...,...
13418,rt safeagain1 russias oil and gas industry wil...,1
5390,rt businessinsider a climate change skeptic is...,1
860,rt bramnessellen we need to tap into the scien...,1
15795,china’s coal use drops showing commitment to c...,2


In [369]:
pro = con[con['sentiment']==1]
news = con[con['sentiment']==2]
neutral = con[con['sentiment']==0]
anti = con[con['sentiment']==-1]

In [370]:
# As usual, we start by importing our modules
from sklearn.utils import resample
# Upsample minority
spam_upsampled1 = resample(anti,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=len(pro), # match number in minority class
                          random_state=42) # reproducible results

# Combine upsampled minority class with majority class
upsampled = pd.concat([spam_upsampled1, pro])

# Check new class counts
upsampled['sentiment'].value_counts()

-1    7635
 1    7635
Name: sentiment, dtype: int64

In [371]:
# As usual, we start by importing our modules
from sklearn.utils import resample
# Upsample minority
spam_upsampled2 = resample(news,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=len(pro), # match number in minority class
                          random_state=27) # reproducible results

# Combine upsampled minority class with majority class
upsampled = pd.concat([spam_upsampled1, spam_upsampled2, pro])

# Check new class counts
upsampled['sentiment'].value_counts()

-1    7635
 2    7635
 1    7635
Name: sentiment, dtype: int64

In [372]:
# As usual, we start by importing our modules
from sklearn.utils import resample
# Upsample minority
spam_upsampled3 = resample(neutral,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=len(pro), # match number in minority class
                          random_state=42) # reproducible results

# Combine upsampled minority class with majority class
upsampled = pd.concat([spam_upsampled1, spam_upsampled2, spam_upsampled3, pro])

# Check new class counts
upsampled['sentiment'].value_counts()

-1    7635
 2    7635
 1    7635
 0    7635
Name: sentiment, dtype: int64

In [373]:
upsampled

,message,sentiment
15202,dino survey by 101 margin americans think al g...,-1
14838,rt pmgeezer how the global warming fraud will ...,-1
4780,rt bittrscrptreadr on a day of total humanitar...,-1
13319,rt realdonaldtrump its freezing outside where ...,-1
4841,the thing is that im not going to deny climate...,-1
...,...,...
5191,rt nytnational trump has called climate change...,1
13418,rt safeagain1 russias oil and gas industry wil...,1
5390,rt businessinsider a climate change skeptic is...,1
860,rt bramnessellen we need to tap into the scien...,1


In [374]:
# model
from sklearn.model_selection import train_test_split

X_train1 = upsampled['message']  
y_train1 = upsampled['sentiment']
X_unseen = df_new['message'] 

In [375]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('lr', LogisticRegression())
                      ])

# Feed the training data through the pipeline
text_clf.fit(X_train1, y_train1) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [376]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train1)

In [377]:
predictions = text_clf.predict(X_test)

In [378]:
y_pred = text_clf.predict(X_unseen)

In [379]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 82  18  23   3]
 [ 29 112  65  18]
 [ 44  83 664 104]
 [  5   8  29 295]]


In [380]:
# classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.51      0.65      0.57       126
           0       0.51      0.50      0.50       224
           1       0.85      0.74      0.79       895
           2       0.70      0.88      0.78       337

    accuracy                           0.73      1582
   macro avg       0.64      0.69      0.66      1582
weighted avg       0.74      0.73      0.73      1582



In [381]:
# print accuracy
print(metrics.accuracy_score(y_test,predictions))

0.7288242730720607


In [382]:
New = df_new.index

In [383]:
New = {'tweetid': New, 'sentiment': np.round(y_pred,0)}
submission = pd.DataFrame(data= New)
submission = submission[['tweetid', 'sentiment']]

In [384]:
submission.set_index('tweetid')

,sentiment
tweetid,
169760,1
35326,1
224985,1
476263,1
872928,0
...,...
895714,1
875167,1
78329,2


In [385]:
submission.head(20)

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0
5,75639,1
6,211536,0
7,569434,2
8,315368,2
9,591733,1


In [386]:
submission.to_csv("kaggle.csv",index  = False) #writing csv file